# Jupyter Notebook to clean install all the contracts

# Set Up Environment

In [1]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'updated20201125_custom_contracts_20201122161857.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [2]:
print(env)

{'iconservice': 'http://18.237.205.52:9000/', 'nid': 3}


In [2]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value
            

In [3]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [4]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [8]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
print(deployer_wallet.get_address())
print(icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"))

hx91bf040426f226b3bfcd2f0b5967bbb0320525ce
9339429034030198414715


#### Check balance

In [6]:

pprint(contracts)

{'addressProvider': {'SCORE': 'cxb6109397557eedc7c140a157f3358c14e80f533a',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cx65b6b77e742e57da956bdab61ede5a395dcc8603',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cx74e824d2da7a0331c9a918dfd0f93fcb89b894ec',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cx62ff612685e8ac85d2d814467c1ccae26d11a367',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxcea1bd2eb07440af158eaa52e58cb06b86681f30',
                             'zip': '/home/elite/Desktop/OMM '
                                    'OG/openmoney

### Importing wallet from keystore file

In [5]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

708876575999000000000000000

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print("deployer wallet:  ",deployer_wallet.get_address())
# caller_wallet = KeyWallet.create()
# print(caller_wallet.get_address())

### Transferring ICX

In [6]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx682d355d1b745a0284d91e4731408d4a53401ccb")\
    .nid(3) \
    .nonce(100) \
    .value(4000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'to': 'hx682d355d1b745a0284d91e4731408d4a53401ccb', 'stepLimit': '0x30d40', 'timestamp': '0x5b4bf3fb8fcde', 'nid': '0x3', 'value': '0xd8d726b7177a800000', 'nonce': '0x64', 'signature': 'rkqAue7Kmim6R6xsP0XvA99uyCloXmv4HvWmPTcbtQskbROjymdaipHp6s+4NegclM2oAqertNmnH7XaTRkXYQA='}


{'txHash': '0xdc894570efc80dab6e745af05afea34402f1c94984693419b8bca22f541659e9',
 'blockHeight': 9543161,
 'blockHash': '0x185e3692e5198172aea3c333ee908ee69c8197d96c15223d452fee2099d34f62',
 'txIndex': 1,
 'to': 'hx682d355d1b745a0284d91e4731408d4a53401ccb',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

### Transfering USDb to a wallet

In [ ]:
params = {"_to": "", "_value": 1000000000000000000000,}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Depositing USDb from a wallet

In [67]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xb1fc5062c7d5fa859a08dd5b505b322bd03450067aec12f06e456c79c25c7d26',
 'blockHeight': 443726,
 'blockHash': '0x1bcfa6746677749da2e16fb7b42ff3708e2dcb3a9fcb7f6a35458f58c72ea4e6',
 'txIndex': 1,
 'to': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
 'stepUsed': 729440,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 729440,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x0',
    '0x0'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x3635c9adc5dea00000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4

# Borrowing USDb

In [75]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":150*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02', 'stepLimit': '0x11a350', 'timestamp': '0x5b3ab99c871a5', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx2c002610b8aeaff84f6cb97c28294b1986016143', '_amount': '0x821ab0d4414980000'}}, 'signature': 'nnBAhBv6BGeWZE63Rr6d02x+c/R36yjf49WvR/G5VLd6hz7tKN6lL6lG/kghrjVg0ZPWIf2KdJi+rmBPoRrFWAA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4f24f998ab722bbcdd6589d23f1bd67cc98a4f30ff863710b291f60545017e0f',
 'blockHeight': 443982,
 'blockHash': '0x795f4716b2053eca2b1b8585c02c10fe30705843841760035bf91b80abb581c9',
 'txIndex': 1,
 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02',
 'stepUsed': 1055920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1055920,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x864ae65482800',
    '0x3e2c284391c000'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx879e276135a6f25236a8abf2e4c9801887538f60',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x821ab0d4414980000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x0

# Reedeming/Withdrawing USDb

In [ ]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Repaying USDb

In [ ]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 100000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 100000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Depositing ICX

In [ ]:
params = {"_amount": 500 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']['SCORE']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(500 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Withdrawing ICX 

In [ ]:
params = {"_amount": 250 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX']['SCORE'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Borrow sICX

In [ ]:
params ={"_reserve": contracts['sicx']['SCORE'], "_amount":150 * 10 ** 18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Repay sICX

In [ ]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 100 * 10 ** 18}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 100 * 10 ** 18, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Readonly methods to get data

## Getting specific reserve data

### This returns the reserve data of USDb

In [46]:
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 1900.0,
 'baseLTVasCollateral': 0.6,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': 0.013333333333333332,
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1606384274330287,
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': 0.0006,
 'oTokenAddress': 'cx32294887f083960fb9a03ee8a91c5562f4202120',
 'reserveAddress': 'cxf56e9fb10e8a3df294e0c0ee23128e4d0e7231ab',
 'totalBorrows': 100.0,
 'totalLiquidity': 2000.0,
 'usageAsCollateralEnabled': '0x1'}


### This method returns the reserve data of sICX

In [48]:
params = {'_reserve': contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 500.0,
 'baseLTVasCollateral': 0.33,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': 0.058333333333333334,
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1606381414982641,
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': 0.02625,
 'oTokenAddress': 'cxfbdcc880d70dc3f700717a777c8ebab7fe1ed893',
 'reserveAddress': 'cx58b79cb2ed3602f7d71f2a9035d3032620318799',
 'totalBorrows': 500.0,
 'totalLiquidity': 1000.0,
 'usageAsCollateralEnabled': '0x1'}


## Getting data of all reserves

### This returns the reserve data of all the reserves

In [37]:
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': 2000.0,
          'baseLTVasCollateral': 0.6,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': 0.01,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': 1606379856315421,
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 0.65,
          'liquidityCumulativeIndex': 1.0,
          'liquidityRate': '0x0',
          'oTokenAddress': 'cx32294887f083960fb9a03ee8a91c5562f4202120',
          'reserveAddress': 'cxf56e9fb10e8a3df294e0c0ee23128e4d0e7231ab',
          'totalBorrows': '0x0',
          'totalLiquidity': 2000.0,
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': 500.0,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': 0.058333333333333334,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1

## Getting user account data

### Returns the user data of all the reserves combined

In [49]:
params = {'_user': 'hx455dfffb56126d29d3542da8cbaabb865af190a0'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 0.65,
 'currentLtv': 0.33,
 'healthFactor': 3.241903299041176,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 100.00001014713393,
 'totalCollateralBalanceUSD': 500.0012901921613,
 'totalFeesUSD': 0.25,
 'totalLiquidityBalanceUSD': 500.0012901921613}


### Getting user reserve data for USDb reserve

In [51]:
params = {'_user': 'hx3ebaf06e2ee517155c879692461872f51b7b1e17' ,"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.013333333333333332,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 2000.0000173896499,
 'currentOTokenBalanceUSD': 2000.0000173896499,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0006,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 0.0}


### Getting user reserve data for sICX reserve

In [53]:
params = {'_user': 'hx3ebaf06e2ee517155c879692461872f51b7b1e17' ,"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.013333333333333332,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 2000.0000195966516,
 'currentOTokenBalanceUSD': 2000.0000195966516,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0006,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 0.0}


### Getting the data of user for all reserves

In [27]:
params = {'_user': 'hx3ebaf06e2ee517155c879692461872f51b7b1e17'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.01,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 2000.0,
          'currentOTokenBalanceUSD': 2000.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 1e-18,
          'userBorrowCumulativeIndex': 0.0},
 'sICX': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0}}


## Get reserve configurations


### Getting configuration of USDb reserve

In [20]:
params = {"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 600000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Getting configuration of sICX reserve

In [17]:
params = {"_reserve":contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)

{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [18]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 600000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'sICX': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [31]:
params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

800224578941786233351189


## Realtime ICX balance of user(with accured interest)

In [32]:
params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))


0
